# Visualizzatore di backup Redis

Questo notebook consente di ispezionare i file JSON generati da ``redis_backup``. Carica il payload, ne mostra il riepilogo e prova a decodificare i valori memorizzati come stringhe per presentarli in chiaro.


## Configurazione del percorso

Imposta il percorso del file di backup JSON esportato con ``save_backup_to_file``. Il notebook non esegue alcuna connessione a Redis: tutte le informazioni vengono lette direttamente dal file.


In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Any, Iterable
import json
import pprint

from redis_backup import (
    decode_bytes,
    display_backup_summary,
    load_backup_from_file,
)


In [ ]:
# Percorso del backup da analizzare.
BACKUP_PATH = Path('/percorso/al/tuo/backup.json')

backup = None
if BACKUP_PATH.exists():
    backup = load_backup_from_file(BACKUP_PATH)
    display_backup_summary(backup)
else:
    print("⚠️ Aggiorna BACKUP_PATH con il percorso corretto del file di backup.")


In [ ]:
from dataclasses import dataclass


class DumpDecodeError(RuntimeError):
    """Errore generico per la decodifica di payload DUMP."""


@dataclass
class DumpSections:
    payload: bytes
    version: int
    checksum: bytes


def split_dump_sections(raw: bytes) -> DumpSections:
    """Separa payload, versione RDB e checksum da un dump Redis."""

    if len(raw) < 10:
        raise DumpDecodeError("Payload DUMP troppo corto per contenere metadati")
    checksum = raw[-8:]
    version_bytes = raw[-10:-8]
    version = int.from_bytes(version_bytes, "little", signed=False)
    payload = raw[:-10]
    return DumpSections(payload=payload, version=version, checksum=checksum)


class _LengthEncoding:
    __slots__ = ("value", "encoding")

    def __init__(self, value: int | None, encoding: int | None = None) -> None:
        self.value = value
        self.encoding = encoding


RDB_ENCODING_INT8 = 0
RDB_ENCODING_INT16 = 1
RDB_ENCODING_INT32 = 2
RDB_ENCODING_LZF = 3


def _read_length_info(buffer: bytes, offset: int) -> tuple[_LengthEncoding, int]:
    if offset >= len(buffer):
        raise DumpDecodeError("Offset fuori range durante la lettura della lunghezza")
    first = buffer[offset]
    prefix = first >> 6
    if prefix == 0:
        length = first & 0x3F
        return _LengthEncoding(length), offset + 1
    if prefix == 1:
        if offset + 1 >= len(buffer):
            raise DumpDecodeError("Lunghezza codificata su 14 bit troncata")
        second = buffer[offset + 1]
        length = ((first & 0x3F) << 8) | second
        return _LengthEncoding(length), offset + 2
    if prefix == 2:
        if offset + 4 >= len(buffer):
            raise DumpDecodeError("Lunghezza codificata su 32 bit troncata")
        length = int.from_bytes(buffer[offset + 1 : offset + 5], "big", signed=False)
        return _LengthEncoding(length), offset + 5
    return _LengthEncoding(None, first & 0x3F), offset + 1


def lzf_decompress(data: bytes, expected_length: int) -> bytes:
    """Implementazione minimale della decompressione LZF usata da Redis."""

    output = bytearray()
    idx = 0
    data_len = len(data)
    while idx < data_len:
        ctrl = data[idx]
        idx += 1
        if ctrl < 32:
            literal_len = ctrl + 1
            if idx + literal_len > data_len:
                raise DumpDecodeError("Sequenza LZF letterale troncata")
            output.extend(data[idx : idx + literal_len])
            idx += literal_len
        else:
            length = ctrl >> 5
            ref_offset = len(output) - ((ctrl & 0x1F) << 8) - 1
            if length == 7:
                if idx >= data_len:
                    raise DumpDecodeError("Sequenza LZF troncata durante l'estensione della lunghezza")
                length += data[idx]
                idx += 1
            if idx >= data_len:
                raise DumpDecodeError("Sequenza LZF troncata durante il calcolo del riferimento")
            ref_offset -= data[idx]
            idx += 1
            length += 2
            if ref_offset < 0:
                raise DumpDecodeError("Riferimento LZF negativo")
            for _ in range(length):
                if ref_offset >= len(output):
                    raise DumpDecodeError("Riferimento LZF fuori range")
                output.append(output[ref_offset])
                ref_offset += 1
    if len(output) != expected_length:
        raise DumpDecodeError(
            f"Lunghezza decompressa inattesa: atteso {expected_length}, ottenuto {len(output)}"
        )
    return bytes(output)


def _decode_special_encoding(buffer: bytes, offset: int, encoding: int) -> tuple[bytes, int]:
    if encoding == RDB_ENCODING_INT8:
        if offset >= len(buffer):
            raise DumpDecodeError("Intero codificato su 8 bit troncato")
        value = int.from_bytes(buffer[offset : offset + 1], "little", signed=True)
        return str(value).encode("ascii"), offset + 1
    if encoding == RDB_ENCODING_INT16:
        if offset + 1 >= len(buffer):
            raise DumpDecodeError("Intero codificato su 16 bit troncato")
        value = int.from_bytes(buffer[offset : offset + 2], "little", signed=True)
        return str(value).encode("ascii"), offset + 2
    if encoding == RDB_ENCODING_INT32:
        if offset + 3 >= len(buffer):
            raise DumpDecodeError("Intero codificato su 32 bit troncato")
        value = int.from_bytes(buffer[offset : offset + 4], "little", signed=True)
        return str(value).encode("ascii"), offset + 4
    if encoding == RDB_ENCODING_LZF:
        length_info, offset = _read_length_info(buffer, offset)
        if length_info.value is None:
            raise DumpDecodeError("Lunghezza compressa non valida")
        compressed_len = length_info.value
        length_info, offset = _read_length_info(buffer, offset)
        if length_info.value is None:
            raise DumpDecodeError("Lunghezza originale non valida")
        uncompressed_len = length_info.value
        end = offset + compressed_len
        if end > len(buffer):
            raise DumpDecodeError("Dati LZF compressi troncati")
        chunk = buffer[offset:end]
        offset = end
        return lzf_decompress(chunk, uncompressed_len), offset
    raise DumpDecodeError(f"Codifica speciale non supportata: {encoding}")


def _read_encoded_string(buffer: bytes, offset: int) -> tuple[bytes, int]:
    length_info, offset = _read_length_info(buffer, offset)
    if length_info.value is not None:
        length = length_info.value
        end = offset + length
        if end > len(buffer):
            raise DumpDecodeError("Stringa codificata troncata")
        return buffer[offset:end], end
    if length_info.encoding is None:
        raise DumpDecodeError("Codifica stringa sconosciuta")
    return _decode_special_encoding(buffer, offset, length_info.encoding)


def decode_string_from_dump(raw: bytes) -> bytes:
    sections = split_dump_sections(raw)
    payload = sections.payload
    if not payload:
        raise DumpDecodeError("Payload vuoto")
    object_type = payload[0]
    if object_type != 0:
        raise DumpDecodeError(f"Tipo di oggetto non stringa: {object_type}")
    value, offset = _read_encoded_string(payload, 1)
    if offset != len(payload):
        # Ignora eventuali byte addizionali non previsti
        value = value
    return value


def decode_key(entry: dict[str, Any]) -> bytes:
    return decode_bytes(entry["key"])


def text_preview(value: bytes, *, limit: int = 120) -> str:
    text = value.decode("utf-8", errors="replace")
    if len(text) > limit:
        return text[: limit - 1] + "…"
    return text


def try_decode_value(entry: dict[str, Any]) -> tuple[str, dict[str, Any]]:
    value_info = dict(entry.get("value") or {})
    data_b64 = value_info.get("data")
    if not data_b64:
        return "<nessun valore>", value_info
    raw = decode_bytes(data_b64)
    details: dict[str, Any] = {
        "dump_size": len(raw),
    }
    try:
        sections = split_dump_sections(raw)
        details["rdb_version"] = sections.version
        details["checksum"] = sections.checksum.hex()
    except DumpDecodeError as exc:
        details["errore_dump"] = str(exc)
        return "<dump non valido>", details
    if entry.get("type") == "string":
        try:
            decoded = decode_string_from_dump(raw)
        except DumpDecodeError as exc:
            details["errore_decodifica"] = str(exc)
            return "<stringa non decodificata>", details
        details["decoded_bytes"] = decoded
        preview = text_preview(decoded)
        return preview, details
    return f"<{entry.get('type')} - {len(sections.payload)} byte>", details


def summarise_entries(entries: Iterable[dict[str, Any]], limit: int = 20) -> list[dict[str, Any]]:
    summary: list[dict[str, Any]] = []
    for index, entry in enumerate(entries):
        if index >= limit:
            break
        key_bytes = decode_key(entry)
        value_preview, _ = try_decode_value(entry)
        summary.append(
            {
                "key": text_preview(key_bytes),
                "type": entry.get("type"),
                "ttl_ms": entry.get("pttl"),
                "value_preview": value_preview,
            }
        )
    return summary


def find_entry(entries: Iterable[dict[str, Any]], key: bytes | str) -> dict[str, Any]:
    if isinstance(key, str):
        key_bytes = key.encode("utf-8")
    else:
        key_bytes = key
    for entry in entries:
        if decode_key(entry) == key_bytes:
            return entry
    raise KeyError(key)


def inspect_entry(entry: dict[str, Any]) -> None:
    key_bytes = decode_key(entry)
    print(f"Chiave: {key_bytes!r}")
    print(f"Tipo Redis: {entry.get('type')}")
    ttl = entry.get("pttl")
    print(f"TTL (ms): {ttl if ttl is not None else 'persistente'}")
    preview, details = try_decode_value(entry)
    print(f"Anteprima valore: {preview}")
    print("Dettagli:")
    pprint.pprint(details)
    decoded = details.get("decoded_bytes")
    if isinstance(decoded, (bytes, bytearray)):
        text = decoded.decode("utf-8", errors="replace")
        print("
Contenuto in chiaro:")
        print(text)
        stripped = text.strip()
        if stripped.startswith("{") and stripped.endswith("}"):
            try:
                parsed = json.loads(text)
            except json.JSONDecodeError:
                pass
            else:
                print("
JSON decodificato:")
                pprint.pprint(parsed)


In [ ]:
if backup is not None:
    entries = list(backup.get('entries', []))
    print(f'Chiavi totali nel backup: {len(entries)}')
    preview_rows = summarise_entries(entries, limit=20)
    try:
        import pandas as pd  # type: ignore
    except ModuleNotFoundError:
        pd = None
    if preview_rows:
        if 'pd' in locals() and pd is not None:
            display(pd.DataFrame(preview_rows))
        else:
            for row in preview_rows:
                print(row)
    else:
        print('Nessuna chiave nel backup selezionato.')


## Ispezione di una chiave specifica

Utilizza ``find_entry`` per recuperare una chiave dal backup e ``inspect_entry`` per mostrarne i dettagli e il contenuto (quando possibile).


In [ ]:
if backup is not None:
    # Esempio: sostituisci 'nome:della:chiave' con la chiave che vuoi analizzare.
    try:
        entry = find_entry(entries, 'nome:della:chiave')
    except KeyError:
        print('Chiave non trovata: aggiorna il nome per visualizzare il contenuto.')
    else:
        inspect_entry(entry)
